In [144]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_rows', None)
import numpy as np

In [160]:
df = pd.read_csv('PORTO BELO.csv', header=None, dtype=str)
df.columns = ['CNPJ', 'TIPO', 'NOME EMPRESARIAL', 'NOME FANTASIA', '(?)', 'ABERTURA', '(?)', '(?)','(?)','(?)', 'COD. (NATUREZA JURIDICA)', 'DATA SIT. CADASTRAL', 'COD. (ATIVIDADE)', 'AV/RUA', 'ENDERECO', 'NUMERO', 'DESC. ENDERECO', 'BAIRRO','CEP', 'ESTADO', '(?)', 'CIDADE', 'PREFIXO_1', 'TELEFONE_1', 'PREFIXO_2', 'TELEFONE_2', '(?)', '(?)', 'EMAIL', '(?)', '(?)', '(?)', '(?)', '(?)', '(?)','(?)', '(?)', '(?)']
df = df.drop('(?)', axis=1)

In [161]:
# IRA JUNTAR OS DADOS DE PREFIXO E TELEFONE EM UMA UNICA COLUNA, FAZER ISSO PARA OS FONE_1 E FONE_2
for row, content in df.T.iteritems():
    fone1 = str(content['PREFIXO_1'])  + str(content['TELEFONE_1'])
    if fone1 == 'nannan':
        df.loc[row, 'FONE_1'] = np.NaN
    else:
        df.loc[row, 'FONE_1'] = fone1
        
df = df.drop(['PREFIXO_1', 'TELEFONE_1'], axis=1)


for row, content in df.T.iteritems():
    fone2 = str(content['PREFIXO_2'])  + str(content['TELEFONE_2'])
    if fone2 == 'nannan':
        df.loc[row, 'FONE_2'] = np.NaN
    else:
        df.loc[row, 'FONE_2'] = fone2
        
df = df.drop(['PREFIXO_2', 'TELEFONE_2'], axis=1)

In [162]:
# IRA JUNTAR OS DADOS DE AV/RUA, ENDERECO E NUMERO EM UMA MESMA COLUNA
for row, content in df.T.iteritems():
    if str(content['DESC. ENDERECO']) != 'nan':
        address =  str(content['AV/RUA']) + ' ' + str(content['ENDERECO']) + ', ' + str(content['NUMERO']) + ' (' + str(content['DESC. ENDERECO']) + ')'
    else:
        address =  str(content['AV/RUA']) + ' ' + str(content['ENDERECO']) + ', ' + str(content['NUMERO'])
    
    df.loc[row, 'FULL ADDRESS'] = address

df = df.drop(['AV/RUA', 'ENDERECO', 'NUMERO', 'DESC. ENDERECO'], axis=1)

In [163]:
def formatDate(old_date):
    ''' formata a data de string padrao USA para padrao BR com divisores'''
    
    year = old_date[0:4]
    month = old_date[4:6]
    day = old_date[6:]
    return day + '/' + month + '/' + year

In [164]:
# ALTERA O FORMATO DAS DATAS
for row, content in df.T.iteritems():
    df.loc[row, 'ABERTURA'], df.loc[row, 'DATA SIT. CADASTRAL'] =  formatDate(content['ABERTURA']), formatDate(content['DATA SIT. CADASTRAL'])

In [165]:
# deleta empresas que nao possuem telefone nem email
unuseful = df[df['FONE_1'].isnull() & df['FONE_2'].isnull() & df['EMAIL'].isnull()].index
df.drop(unuseful, inplace=True)

,CNPJ,MATRIZ/FILIAL,NOME EMPRESARIAL,NOME FANTASIA,ABERTURA,COD. (NATUREZA JURIDICA),DATA SIT. CADASTRAL,COD. (ATIVIDADE),BAIRRO,CEP,ESTADO,CIDADE,EMAIL,FONE_1,FONE_2,FULL ADDRESS
0,00000000361372,2,BANCO DO BRASIL SA,NaN,03/11/2005,2038,23/05/1988,6421200,CENTRO,88210000,SC,PORTO BELO,age3272@bb.com.br,4733697200,NaN,"RUA CAP. GUALBERTO LEAL NUNES, 112"
1,00000000361372,2,BANCO DO BRASIL SA,NaN,03/11/2005,2038,23/05/1988,6421200,CENTRO,88210000,SC,PORTO BELO,age3272@bb.com.br,4733697200,NaN,"RUA CAP. GUALBERTO LEAL NUNES, 112"
2,00000000608017,2,BANCO DO BRASIL SA,PORTO BELO / BESC (SC),11/07/2008,2038,11/07/2008,6422100,CENTRO,88210000,SC,PORTO BELO,supersc@bb.com.br,4832393000,NaN,"AVENIDA GOVERNADOR CELSO RAMOS, 2222"
3,00000000608017,2,BANCO DO BRASIL SA,PORTO BELO / BESC (SC),11/07/2008,2038,11/07/2008,6422100,CENTRO,88210000,SC,PORTO BELO,supersc@bb.com.br,4832393000,NaN,"AVENIDA GOVERNADOR CELSO RAMOS, 2222"
4,00067871000122,1,RODRIGUES & BARRETO LTDA,ARTE METAL JOALHEIROS,28/08/2018,2062,16/05/1994,3211602,CENTRO,88210000,SC,PORTO BELO,NaN,NaN,NaN,"RUA JOSE GUERREIRO FILHO, 294"


In [195]:
#DEFINE QUAIS SAO MATRIZES E QUAIS SAO FILIAIS
for row, content in df.T.iteritems():
    if df.loc[row, 'MATRIZ/FILIAL'] == '1':
        df.loc[row, 'MATRIZ/FILIAL'] = 'MATRIZ'
    else:
        df.loc[row, 'MATRIZ/FILIAL'] = 'FILIAL'

In [196]:
df.head()

,CNPJ,MATRIZ/FILIAL,NOME EMPRESARIAL,NOME FANTASIA,ABERTURA,COD. (NATUREZA JURIDICA),DATA SIT. CADASTRAL,COD. (ATIVIDADE),BAIRRO,CEP,ESTADO,CIDADE,EMAIL,FONE_1,FONE_2,FULL ADDRESS
0,00000000361372,FILIAL,BANCO DO BRASIL SA,NaN,03/11/2005,2038,23/05/1988,6421200,CENTRO,88210000,SC,PORTO BELO,age3272@bb.com.br,4733697200,NaN,"RUA CAP. GUALBERTO LEAL NUNES, 112"
1,00000000361372,FILIAL,BANCO DO BRASIL SA,NaN,03/11/2005,2038,23/05/1988,6421200,CENTRO,88210000,SC,PORTO BELO,age3272@bb.com.br,4733697200,NaN,"RUA CAP. GUALBERTO LEAL NUNES, 112"
2,00000000608017,FILIAL,BANCO DO BRASIL SA,PORTO BELO / BESC (SC),11/07/2008,2038,11/07/2008,6422100,CENTRO,88210000,SC,PORTO BELO,supersc@bb.com.br,4832393000,NaN,"AVENIDA GOVERNADOR CELSO RAMOS, 2222"
3,00000000608017,FILIAL,BANCO DO BRASIL SA,PORTO BELO / BESC (SC),11/07/2008,2038,11/07/2008,6422100,CENTRO,88210000,SC,PORTO BELO,supersc@bb.com.br,4832393000,NaN,"AVENIDA GOVERNADOR CELSO RAMOS, 2222"
8,00071031000133,MATRIZ,ROCHA & TOMASI FLORICULTURA LTDA,STONE GARDEN,03/11/2005,2062,19/05/1994,4789002,PEREQUE,88210000,SC,PORTO BELO,NaN,4733486061,NaN,"RODOVIA SC 412, 1498"


In [179]:
# falta tratar o codigo da atividade e codigo da natureza juridica
# salvar em csv para celular
# salar em um banco de dados

10702

In [ ]:
#criar verificador de whatsapp